## This is for change YOLO format .txt file to .json file , but recommend to use the latest version below.

In [1]:
import os
import json
from PIL import Image

# Define function to read txt files
def readFile(filename):
    cata = []
    xm = []
    ym = []
    w = []
    h = []
    score = []
    with open(filename, encoding='utf-8') as lines:
        for line in lines:
            cata.append(line.split(" ")[0])
            xm.append(line.split()[1])
            ym.append(line.split(" ")[2])
            w.append(line.split(" ")[3])
            h.append(line.split(" ")[4])
            score.append(line.split(" ")[5].strip())
    return cata, xm, ym, w, h, score

# Define function to process images and generate JSON files
def process_images(input_folder):
    # Loop through each image in the input folder
    for filename in os.listdir(input_folder):
        if filename.endswith('.txt'):
            txt_path = os.path.join(input_folder, filename)
            image_filename = filename.split('.')[0] + '.png'
            img_path = os.path.join(input_folder, image_filename)

            if os.path.exists(img_path):
                img = Image.open(img_path)
                imgSize = img.size
                dw = imgSize[0]
                dh = imgSize[1]

                d_1, d_2, d_3, d_4, d_5, d_6 = readFile(txt_path)

                # Store each result as a dictionary and then append each dictionary to a list
                result = []
                # Loop through the current txt file
                for i in range(len(d_1)):
                    # Label corresponds to the target (0 is fire, 1 is smoke)
                    if int(d_1[i]):
                        d_7 = 'smoke'
                    else:
                        d_7 = 'fire'
                    dic = {'name': image_filename, 'category': d_7, 'bbox': [abs(round(((float(d_2[i]))*dw-(0.5*(float(d_4[i]))*dw)), 2)), abs(round(((float(d_3[i]))*dh-(0.5*(float(d_5[i]))*dh)), 2)), abs(round(((float(d_2[i]))*dw+(0.5*(float(d_4[i]))*dw)), 2)), abs(round(((float(d_3[i]))*dh+(0.5*(float(d_5[i]))*dh)), 2))], 'score': (d_6[i])}
                    result.append(dic)

                # Construct the output file path
                output_dir = os.path.dirname(txt_path)
                newFile = os.path.join(output_dir, 'Submission.json')

                # Write the content into the file
                with open(newFile, 'a') as name:
                    for i in range(len(result)):
                        jsonData = json.dumps(result[i], indent=4)
                        # Write into the file
                        name.write(jsonData+','+'\n')

            else:
                abc = []
                dic = {'name': image_filename, 'category': None, 'bbox': None, 'score': None}
                abc.append(dic)

                with open(newFile, 'a') as name:
                    for i in range(len(abc)):
                        jsonData = json.dumps(abc[i], indent=4)
                        # Write into the file
                        name.write(jsonData+','+'\n')

# Define input folder
# Use relative path
input_folder = "project_agritue_dataset_2024_03_22_15_01_06_segmentation mask 1.1/SegmentationClass"  # Enter folder path

# Execute the processing function
process_images(input_folder)
print("Finished!")



Finished!


## The latest version !!! Can used for every mask picture png. to generate .json file

In [7]:
import copy
import cv2
import os
import shutil
import numpy as np

# Path: Directory containing segmentation masks
# Using relative path
path = "project_agritue_dataset_2024_03_22_15_01_06_segmentation mask 1.1/predict_mask"
files = os.listdir(path)

# Iterate through each file in the directory
for file in files:
    name = file.split('.')[0]
    file_path = os.path.join(path, name+'.png')
    img = cv2.imread(file_path)
    H, W = img.shape[0:2]  # Get the height and width of the image
    print(H, W)

    # Convert the image to grayscale
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # Apply Otsu's thresholding to get a binary image
    ret, bin_img = cv2.threshold(gray_img, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    # Find contours in the binary image
    cnt, hit = cv2.findContours(bin_img, cv2.RETR_TREE, cv2.CHAIN_APPROX_TC89_KCOS)

    # Open a text file to write the contour coordinates
    # Using relative path
    f = open("project_agritue_dataset_2024_03_22_15_01_06_segmentation mask 1.1/predict_mask/{}.json".format(file.split(".")[0]), "a+")
    f.write("{\n")
    f.write('  "type": "FeatureCollection",\n')
    f.write('  "features": [\n')

    for index, j in enumerate(cnt):
        result = []
        for i in j:
            temp = list(i[0])
            temp[0] /= W  # Normalize x-coordinate
            temp[1] /= H  # Normalize y-coordinate
            result.append(temp)

        # If the contour is not empty, save it as a closed polygon
        if result:
            result.append(result[0])  # Add the starting point to close the polygon

            # Write the contour coordinates into the JSON file
            f.write('    {\n')
            f.write('      "type": "Feature",\n')
            f.write('      "properties": {\n')
            f.write('        "Name": "contour{}"\n'.format(index))
            f.write('      },\n')
            f.write('      "geometry": {\n')
            f.write('        "type": "Polygon",\n')
            f.write('        "coordinates": [\n')
            f.write('          [\n')

            for idx, line in enumerate(result):
                f.write('            [\n')
                f.write('              {},\n'.format(line[0]))
                f.write('              {}\n'.format(line[1]))
                f.write('            ]{}\n'.format(',' if idx < len(result) - 1 else ''))

            f.write('          ]\n')
            f.write('        ]\n')
            f.write('      }\n')
            if index < len(cnt) - 1:
                f.write('    },\n')  
            else:
                f.write('    }\n')  

    f.write('  ]\n')
    f.write('}\n')
    f.close()

640 640
